## Question

In this exercise, you will learn to change the Signal values in the lower one-third of the ‘Ret’ column to -1 and at the closing time to 0.

Note: The closing time for Indian markets is 15:30 hours.
Instructions

In this exercise, you are required to code 2 lines:
1. Change all Signal values to ‘-1', when returns, ‘Ret’, lie in the quantile less than 0.34.

Hint: Check out the line just above your work area, that changes the ‘Signal’ values to ‘1’ for returns lying in quantiles greater than 0.66.
 
2. Change all Signal values at the time 15 hours and 29 minutes to '0'.
To do this, on column ‘Time’ of the data frame 'Df', apply functions '.dt.hour' and '.dt.minute' to locate the position to be changed.

In [ ]:
# Importing libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt

# Importing data
Df = pd.read_csv('ICICI Minute Data.csv')

Df = Df.drop(Df[Df['Volume']==0].index)
Df['Time']=pd.to_datetime(Df['Time'])

# Splitting data
t=.8
split = int(t*len(Df))

# Creating indicators
n=10

Df['RSI'] = ta.RSI(np.array(Df['Close'].shift(1)), timeperiod=n)
Df['SMA']= Df['Close'].shift(1).rolling(window=n).mean()
Df['Corr']= Df['Close'].shift(1).rolling(window=n).corr(Df['SMA'].shift(1))
Df['SAR']=ta.SAR(np.array(Df['High'].shift(1)),np.array(Df['Low'].shift(1)),0.2,0.2)
Df['ADX']=ta.ADX(np.array(Df['High'].shift(1)),np.array(Df['Low'].shift(1)),np.array(Df['Open']), timeperiod =n)

Df['open'] = Df['Open'].shift(1)
Df['close'] = Df['Close'].shift(1)
Df['high'] = Df['High'].shift(1)
Df['low'] = Df['Low'].shift(1)
Df['OO'] = Df['Open']-Df['open']
Df['OC'] = Df['Open']-Df['close']

Df['Ret'] = (Df['Open'].shift(-1)-Df['Open'])/Df['Open']
for i in range(1,n):
    Df['return%i'%i] = Df['Ret'].shift(i)

# Trimming the data
Df.loc[Df['Corr']<-1,'Corr']=-1
Df.loc[Df['Corr']>1,'Corr']=1
Df=Df.dropna()

# Creating output signal
Df['Signal']=0
Df.loc[Df['Ret']>Df['Ret'][:split].quantile(q=0.66),'Signal']=1

#####Type your code below ######

## Answer

In [ ]:
Df.loc[Df['Ret']<Df['Ret'][:split].quantile(q=0.34),'Signal']=-1
Df.loc[(Df['Time'].dt.hour==15) & (Df['Time'].dt.minute==29), 'Signal']=02

## Explanation

What this does:
	•	Df['Ret'][:split]: uses training portion of the data
	•	.quantile(0.34): finds the 34th percentile (lower third)
	•	Df['Ret'] < ...: filters returns below this threshold
	•	'Signal'] = -1: marks them as sell signals

“If the return is in the lowest third, we predict a price drop → set signal to -1.”

	•	Df['Time'].dt.hour == 15 and .dt.minute == 29 → selects time 15:29
	•	'Signal'] = 0: sets it to neutral

"At market close, we don’t want to hold a position, so we reset the signal."